In [ ]:
import pandas as pd
import numpy as np

In [ ]:
!pip install -q openai

In [ ]:
import os
import json
import time
import logging
import pandas as pd
from typing import Optional, Dict, List
from tqdm.auto import tqdm
from openai import OpenAI, APIError, RateLimitError, APITimeoutError
from kaggle_secrets import UserSecretsClient

class Config:
    INPUT_PARQUET = "/kaggle/input/avito-data/data/test_part_0001.snappy.parquet"
    OUTPUT_DATASET = "avito_train_dataset.jsonl"
    LOG_FILE = "generation_process.log"
    
    SAMPLE_SIZE = 1500
    MIN_LEN = 30
    MAX_LEN = 150
    
    MODEL_NAME = "deepseek/deepseek-v3.2-speciale"
    API_BASE = "https://openrouter.ai/api/v1"
    
    TEMPERATURE = 0.9
    TOP_P = 0.9
    FREQ_PENALTY = 0.1
    PRESENCE_PENALTY = 0.1
    REP_PENALTY = 1.05
    REASONING_EFFORT = "low"
    
    MAX_RETRIES = 5
    RETRY_DELAY = 2

# --- LOGGING SETUP ---
for handler in logging.root.handlers[:]:
    logging.root.removeHandler(handler)

log_formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')

file_handler = logging.FileHandler(Config.LOG_FILE, mode='a', encoding='utf-8')
file_handler.setFormatter(log_formatter)
file_handler.setLevel(logging.INFO)

console_handler = logging.StreamHandler()
console_handler.setFormatter(log_formatter)
console_handler.setLevel(logging.INFO)

logger = logging.getLogger()
logger.setLevel(logging.INFO)
logger.addHandler(file_handler)
logger.addHandler(console_handler)

logger.info("Инициализация скрипта генерации запущена.")

try:
    user_secrets = UserSecretsClient()
    api_key = user_secrets.get_secret("OPENROUTER_API_KEY")
    client = OpenAI(base_url=Config.API_BASE, api_key=api_key)
    logger.info("OpenAI client успешно настроен.")
except Exception as e:
    logger.error(f"Не удалось получить API Key из Secrets: {e}")
    raise e

In [ ]:
def load_and_filter_data():
    logger.info("Загрузка данных из Parquet...")
    
    PREVIOUS_PROGRESS_PATH = "/kaggle/input/avito-700-samples/avito_train_dataset-united.jsonl"
    
    cols = ["base_item_id", "base_title", "base_description", 
            "base_category_name", "base_subcategory_name"]
    
    df = pd.read_parquet(Config.INPUT_PARQUET, columns=cols)
    
    df['desc_len'] = df['base_description'].str.len().fillna(0)
    filtered_df = df[
        (df['desc_len'] >= Config.MIN_LEN) & 
        (df['desc_len'] <= Config.MAX_LEN)
    ].copy()
    
    processed_ids = set()
    
    if os.path.exists(PREVIOUS_PROGRESS_PATH):
        logger.info(f"Считываем ID из загруженного датасета: {PREVIOUS_PROGRESS_PATH}")
        with open(PREVIOUS_PROGRESS_PATH, 'r', encoding='utf-8') as f:
            for line in f:
                try:
                    record = json.loads(line)
                    processed_ids.add(record.get('id'))
                except: continue
                
    if os.path.exists(Config.OUTPUT_DATASET):
        logger.info(f"Считываем ID из локального файла: {Config.OUTPUT_DATASET}")
        with open(Config.OUTPUT_DATASET, 'r', encoding='utf-8') as f:
            for line in f:
                try:
                    record = json.loads(line)
                    processed_ids.add(record.get('id'))
                except: continue
    
    logger.info(f"Всего уже обработано ранее: {len(processed_ids)} записей.")
    
    filtered_df = filtered_df[~filtered_df['base_item_id'].isin(processed_ids)]
    
    remaining_needed = Config.SAMPLE_SIZE - len(processed_ids)
    if remaining_needed <= 0:
        logger.info("Цель по количеству записей уже достигнута!")
        return pd.DataFrame()
        
    if len(filtered_df) > remaining_needed:
        filtered_df = filtered_df.sample(remaining_needed, random_state=42)
        
    logger.info(f"Осталось доделать: {len(filtered_df)} записей.")
    return filtered_df

work_df = load_and_filter_data()

In [ ]:
class DescriptionGenerator:
    def __init__(self, client):
        self.client = client
        self.system_prompt = (
            "Ты — ведущий AI-копирайтер платформы Авито. Твоя задача: профессионально отредактировать описание товара, "
            "сделав его привлекательным, структурированным и грамотным.\n\n"
            "СЛЕДУЙ ПРАВИЛАМ СТРОГО:\n"
            "1. ИСПРАВЛЕНИЕ: Исправь все опечатки, пунктуационные и грамматические ошибки исходного текста.\n"
            "2. ФАКТЫ: Оставь все детали из исходного описания. Категорически запрещено выдумывать состояние товара "
            "или детали, которых нет в тексте.\n"
            "3. ЭКСПЕРТНОСТЬ: Добавь 1-2 предложения с общеизвестными преимуществами данной модели товара.\n"
            "4. СТРУКТУРА:\n"
            "   - Разбей текст на логические абзацы.\n"
            "   - Обязательно добавь один маркированный список (через точку '•').\n"
            "   - Лимит эмодзи: строго не более 2 штук.\n"
            "5. СТИЛЬ: Профессиональный, человечный, лаконичный.\n"
            "6. ОГРАНИЧЕНИЕ ДЛИНЫ: Ориентируйся на объем 40–80 слов. "
            "ВАЖНО: Если исходное описание очень короткое, не пытайся растягивать текст «водой» или выдумками. "
            "В таком случае лучше оставить описание коротким (20-40 слов), но честным и полезным.\n\n"
            "Выдавай ТОЛЬКО финальный текст объявления без вводных фраз."
        )

    def _create_user_prompt(self, row):
        return (
            f"Улучши следующее объявление:\n"
            f"Категория: {row['base_category_name']} / {row['base_subcategory_name']}\n"
            f"Заголовок: {row['base_title']}\n"
            f"Исходное описание: {row['base_description']}\n\n"
            "Результат:"
        )

    def generate(self, row) -> Optional[str]:
        prompt = self._create_user_prompt(row)
        
        for attempt in range(Config.MAX_RETRIES):
            try:
                response = self.client.chat.completions.create(
                    model=Config.MODEL_NAME,
                    messages=[
                        {"role": "system", "content": self.system_prompt},
                        {"role": "user", "content": prompt}
                    ],
                    temperature=Config.TEMPERATURE,
                    top_p=Config.TOP_P,
                    frequency_penalty=Config.FREQ_PENALTY,
                    presence_penalty=Config.PRESENCE_PENALTY,
                    extra_body={"reasoning_effort": Config.REASONING_EFFORT},
                    extra_headers={
                        "HTTP-Referer": "https://kaggle.com",
                        "X-Title": "Avito Portfolio Project"
                    }
                )

                # Проверка на пустой ответ
                content = response.choices[0].message.content
                
                if content:
                    return content.strip()
                else:
                    return None
            
            except APITimeoutError:
                logger.warning(f"Timeout ID {row['base_item_id']}. Retry {attempt+1}...")
                time.sleep(2)
            except RateLimitError:
                logger.warning(f"RateLimit ID {row['base_item_id']}. Sleep 5s...")
                time.sleep(5)
            except Exception as e:
                logger.error(f"Error ID {row['base_item_id']}: {e}")
                return None
                
        logger.error(f"Не удалось получить ответ для ID {row['base_item_id']} после всех попыток.")
        return None

In [ ]:
if not work_df.empty:
    generator = DescriptionGenerator(client)
    
    with open(Config.OUTPUT_DATASET, 'a', encoding='utf-8', buffering=1) as f_out:
        
        logger.info("Начинаем генерацию...")
        
        for _, row in tqdm(work_df.iterrows(), total=len(work_df), desc="Generating"):
            
            generated_text = generator.generate(row)
            
            if generated_text:
                record = {
                    "id": row['base_item_id'],
                    "category_context": f"{row['base_category_name']} / {row['base_subcategory_name']}",
                    "title": row['base_title'],
                    "original_description": row['base_description'],
                    "generated_description": generated_text,
                    "instruction": "Улучши описание объявления для Авито, исправь ошибки и добавь структуру."
                }
                
                f_out.write(json.dumps(record, ensure_ascii=False) + "\n")
                
                logger.info(f"\n[ID: {row['base_item_id']}] SUCCESS")
                logger.info(f"ORIGINAL: {row['base_description'][:100]}...")
                logger.info(f"GENERATED: {generated_text[:100]}...")
                
            else:
                logger.error(f"[ID: {row['base_item_id']}] FAILED to generate.")

    logger.info("Обработка завершена.")
else:
    logger.info("Нет данных для обработки.")

In [ ]:
if os.path.exists(Config.OUTPUT_DATASET):
    print("--- ПРИМЕРЫ ИЗ СОЗДАННОГО ДАТАСЕТА ---")
    with open(Config.OUTPUT_DATASET, 'r', encoding='utf-8') as f:
        for i, line in enumerate(f):
            if i >= 3: break
            data = json.loads(line)
            print(f"\nID: {data['id']}")
            print(f"INPUT: {data['original_description']}")
            print("-" * 20)
            print(f"OUTPUT: {data['generated_description']}")
            print("=" * 40)
else:
    print("Файл датасета не найден.")

In [ ]:
final_file = "final_avito_dataset_1500.jsonl"
old_file = "/kaggle/input/avito-processed-data/avito_train_dataset.jsonl"
new_file = "avito_train_dataset.jsonl"

with open(final_file, 'w', encoding='utf-8') as f_out:
    if os.path.exists(old_file):
        with open(old_file, 'r') as f_in:
            f_out.write(f_in.read())
    if os.path.exists(new_file):
        with open(new_file, 'r') as f_in:
            f_out.write(f_in.read())

print(f"Готово! Финальный файл: {final_file}")